In [4]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import time

print("1")

interval = 60*60*1000 #milliseconds

now = int(time.time()) * 1000 #epoch milliseconds
past = now - interval

pattern = '%Y-%m-%dT%H:%M:%S'

print("2")


#ddm metrics:
ddm_index = "atlas_ddm-metrics-20*"


#Set up elasticsearch
es = Elasticsearch(['es-atlas.cern.ch:9203'],timeout=60, use_ssl=True, verify_certs=True, ca_certs='/etc/pki/tls/certs/CERN-bundle.pem', http_auth='roatlas:la2Dbd5rtn3df!sx')

print("3")


query_queued = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
        "timestamp": {
            "gte": past,
            "lte": now,
            "format": "epoch_millis"
            }
        }
    },
        {"exists" : { "field" : "queued-total" }},
        #{"exists" : { "field" : "_source.dst" }},
        #{"term": {"_source.dst": "IN2P3-LPSC"}},
        #{"term" : {"_index" : 'atlas_ddm-metrics-2017-08-17'}},
        #{"term" : {"dst" : "CERN-PROD"}}
    ]
    }
    }
}

print("4")

scroll_queued = scan(es, query=query_queued, index=ddm_index, scroll='5m', timeout='5m', size=100)


queue_dict = {}

for entry in scroll_queued:
    if not counter%10000:  print("Processing event number ",counter)
    counter += 1
    
    source = entry['_source']['src']
    dest = entry['_source']['dst']
    key = source + ":" + dest
    if key not in queue_dict.keys():
        queue_dict[key] = []

    queued = entry['_source']['queued-total']
    timestamp = entry['_source']['timestamp']
    timestamp = int(time.mktime(time.strptime(timestamp,pattern)))*1000
    queue_dict[key].append([queued,timestamp])

print("5")
    
    
print(queue_dict)


1
2
3
4


ConnectionError: ConnectionError((<urllib3.connection.VerifiedHTTPSConnection object at 0x7f53a0ee33c8>, 'Connection to es-atlas.cern.ch timed out. (connect timeout=60)')) caused by: ConnectTimeoutError((<urllib3.connection.VerifiedHTTPSConnection object at 0x7f53a0ee33c8>, 'Connection to es-atlas.cern.ch timed out. (connect timeout=60)'))